In [ ]:
#| default_exp learner.losses
#| default_cls_lvl 3

In [ ]:
#| export
from tsfast.data import *
from fastai.basics import *
import warnings

In [ ]:
from tsfast.datasets import create_dls_test
from tsfast.models import *

In [ ]:
dls = create_dls_test()
model = SimpleRNN(1,1)

## Loss Functions

In [ ]:
#| export
import functools

def ignore_nan(func):
    '''remove nan values from tensors before function execution, reduces tensor to a flat array, apply to functions such as mse'''
    @functools.wraps(func)
    def ignore_nan_decorator(*args, **kwargs):
#         mask = ~torch.isnan(args[-1]) #nan mask of target tensor
#         args = tuple([x[mask] for x in args]) #remove nan values
        mask = ~torch.isnan(args[-1][...,-1]) #nan mask of target tensor
        args = tuple([x[mask,:] for x in args]) #remove nan values
        return func(*args, **kwargs)
    return ignore_nan_decorator

In [ ]:
n = 1000
y_t = torch.ones(32,n,6)
y_t[:,20]=np.nan
y_p = torch.ones(32,n,6)*1.1

In [ ]:
(~torch.isnan(y_t)).shape

torch.Size([32, 1000, 6])

In [ ]:
y_t.shape

torch.Size([32, 1000, 6])

In [ ]:
assert torch.isnan(mse(y_p,y_t))

In [ ]:
#| export
mse_nan = ignore_nan(mse)

In [ ]:
test_close(mse_nan(y_p,y_t),0.01)

In [ ]:
#| export
import functools
import warnings

def float64_func(func):
    '''calculate function internally with float64 and convert the result back'''
    @functools.wraps(func)
    def float64_func_decorator(*args, **kwargs):
        typ = args[0].dtype
        try:
            # Try to use float64 for higher precision
            args = tuple([x.double() if issubclass(type(x),Tensor) else x for x in args])
            return func(*args, **kwargs).type(typ)
        except TypeError as e:
            # If float64 is not supported on this device, warn the user and fall back to float32
            if "doesn't support float64" in str(e):
                warnings.warn(f"Float64 precision not supported on {args[0].device} device. Using original precision. This may reduce numerical accuracy. Error: {e}")
                return func(*args, **kwargs)
            else:
                raise # Re-raise if it's some other error
    return float64_func_decorator

In [ ]:
Learner(dls,model,loss_func=float64_func(nn.MSELoss())).fit(1)

epoch,train_loss,valid_loss,time
0,0.055687,0.058517,00:03


/var/folders/pc/13zbh_m514n1tp522cx9npt00000gn/T/ipykernel_41670/3967170634.py:17: UserWarning: Float64 precision not supported on mps:0 device. Using original precision. This may reduce numerical accuracy. Error: Cannot convert a MPS Tensor to float64 dtype as the MPS framework doesn't support float64. Please use float32 instead.
  warnings.warn(f"Float64 precision not supported on {args[0].device} device. Using original precision. This may reduce numerical accuracy. Error: {e}")


In [ ]:
#| export
def SkipNLoss(fn,n_skip=0):
    '''Loss-Function modifier that skips the first n samples of sequential data'''
    @functools.wraps(fn)
    def _inner( input, target):
        return fn(input[:,n_skip:].contiguous(),target[:,n_skip:].contiguous())
    
    return _inner

In [ ]:
Learner(dls,model,loss_func=SkipNLoss(nn.MSELoss(),n_skip=30)).fit(1)

epoch,train_loss,valid_loss,time
0,0.048772,0.041046,00:01


In [ ]:
#| export
def CutLoss(fn,l_cut=0,r_cut=None):
    '''Loss-Function modifier that skips the first n samples of sequential data'''
    @functools.wraps(fn)
    def _inner( input, target):
        return fn(input[:,l_cut:r_cut],target[:,l_cut:r_cut])
    
    return _inner

In [ ]:
Learner(dls,model,loss_func=CutLoss(nn.MSELoss(),l_cut=30)).fit(1)

epoch,train_loss,valid_loss,time
0,0.020928,0.015426,00:01


In [ ]:
#| export
def weighted_mae(input, target):
    max_weight = 1.0
    min_weight = 0.1
    seq_len = input.shape[1]

    device = input.device
    if device.type == 'mps':
        # Compute on CPU because MPS does not support logspace yet
        weights = torch.logspace(start=torch.log10(torch.tensor(max_weight)),
                                end=torch.log10(torch.tensor(min_weight)),
                                steps=seq_len, device='cpu').to(device)
        warnings.warn(f"torch.logspace not supported on {device} device. Using cpu. This may reduce numerical performance")
    else:
        # Compute directly on the target device 
        weights = torch.logspace(start=torch.log10(torch.tensor(max_weight)),
                                end=torch.log10(torch.tensor(min_weight)),
                                steps=seq_len, device=device)


    weights = (weights / weights.sum())[None,:,None]

    return ((input-target).abs()*weights).sum(dim=1).mean()


In [ ]:
Learner(dls,model,loss_func=SkipNLoss(weighted_mae,n_skip=30)).fit(1)

epoch,train_loss,valid_loss,time
0,0.076916,0.061655,00:01


/var/folders/pc/13zbh_m514n1tp522cx9npt00000gn/T/ipykernel_41670/691337496.py:13: UserWarning: torch.logspace not supported on mps:0 device. Using cpu. This may reduce numerical performance
  warnings.warn(f"torch.logspace not supported on {device} device. Using cpu. This may reduce numerical performance")


In [ ]:
#| export
def RandSeqLenLoss(fn,min_idx=1,max_idx=None,mid_idx=None):
    '''Loss-Function modifier that truncates the sequence length of every sequence in the minibatch inidiviually randomly.
    At the moment slow for very big batchsizes.'''
    @functools.wraps(fn)
    def _inner( input, target):
        bs,l,_ = input.shape
        if 'max_idx' not in locals():  max_idx = l
        if 'mid_idx' not in locals():  mid_idx = min_idx#+(max_idx-min_idx)//4
        # len_list = torch.randint(min_idx,max_idx,(bs,))
        len_list = np.random.triangular(min_idx,mid_idx,max_idx,(bs,)).astype(int)
        return torch.stack([fn(input[i,:len_list[i]],target[i,:len_list[i]]) for i in range(bs)]).mean()
    return _inner

In [ ]:
Learner(dls,model,loss_func=RandSeqLenLoss(nn.MSELoss())).fit(1)

epoch,train_loss,valid_loss,time
0,0.033088,0.032073,00:09


In [ ]:
#| export
def fun_rmse(inp, targ): 
    '''rmse loss function defined as a function not as a AccumMetric'''
    return torch.sqrt(F.mse_loss(inp, targ))

In [ ]:
Learner(dls,model,loss_func=nn.MSELoss(),metrics=SkipNLoss(fun_rmse,n_skip=30)).fit(1)

epoch,train_loss,valid_loss,fun_rmse,time
0,0.010644,0.010371,0.050435,00:01


In [ ]:
#| export
def cos_sim_loss(inp, targ): 
    '''rmse loss function defined as a function not as a AccumMetric'''
    return (1-F.cosine_similarity(inp,targ,dim=-1)).mean()

In [ ]:
Learner(dls,model,loss_func=cos_sim_loss,metrics=SkipNLoss(fun_rmse,n_skip=30)).fit(1)

epoch,train_loss,valid_loss,fun_rmse,time
0,0.234756,0.244600,0.050515,00:01


In [ ]:
#| export
def cos_sim_loss_pow(inp, targ): 
    '''rmse loss function defined as a function not as a AccumMetric'''
    return (1-F.cosine_similarity(inp,targ,dim=-1)).pow(2).mean()

In [ ]:
Learner(dls,model,loss_func=cos_sim_loss_pow,metrics=SkipNLoss(fun_rmse,n_skip=30)).fit(1)

epoch,train_loss,valid_loss,fun_rmse,time
0,0.444785,0.485400,0.051185,00:01


In [ ]:
#| export
def nrmse(inp, targ): 
    '''rmse loss function scaled by variance of each target variable'''
    mse = (inp-targ).pow(2).mean(dim=[0,1])
    var = targ.var(dim=[0,1])
    return (mse/var).sqrt().mean()

In [ ]:
dls.one_batch()[0].shape

torch.Size([64, 100, 1])

In [ ]:
Learner(dls,model,loss_func=nn.MSELoss(),metrics=SkipNLoss(nrmse,n_skip=30)).fit(1)

epoch,train_loss,valid_loss,nrmse,time
0,0.010247,0.009836,0.176671,00:01


In [ ]:
#| export
def nrmse_std(inp, targ): 
    '''rmse loss function scaled by standard deviation of each target variable'''
    mse = (inp-targ).pow(2).mean(dim=[0,1])
    var = targ.std(dim=[0,1])
    return (mse/var).sqrt().mean()

In [ ]:
Learner(dls,model,loss_func=nn.MSELoss(),metrics=SkipNLoss(nrmse_std,n_skip=30)).fit(1)

epoch,train_loss,valid_loss,nrmse_std,time
0,0.009845,0.009637,0.083519,00:01


In [ ]:
#| export
def mean_vaf(inp,targ):
    return (1-((targ-inp).var()/targ.var()))*100

In [ ]:
Learner(dls,model,loss_func=nn.MSELoss(),metrics=SkipNLoss(mean_vaf,n_skip=30)).fit(1)

epoch,train_loss,valid_loss,mean_vaf,time
0,0.009872,0.009322,98.075012,00:01


In [ ]:
#| include: false
import nbdev; nbdev.nbdev_export()